In [6]:
import numpy as np
# this function simulate 1 batch of K number of X_n, given fixed Lambda, P, Alpha
# @ param N: is the number of total population
# @ param total_time: the time that we want to simulate the process (6 month)
# @ param x_0: initial infected patients (greater than 0)
def simulation_infected(la, p, al, N, total_time, x_0, K):
    # initialize time and infected patient
    t_0 = 0
    # bind variables to initial conditions
    x_n = x_0
    assert(x_n!=0)
    t_n = t_0
    # initiate X_n process and corresponding T_n process
    X_n = [x_0]
    T_n = [t_0]
    # converge
    k = 0
    # loop
    # break condition
    # @ condition1: when the time exceeds the total time
    # @ condition2: when the infected patients go to zero
    # @ condition3: when the whole populations are infected! :(
    while (t_n<total_time and
           x_n != 0 and
           x_n < N and
           k < K
          ):
        i = x_n
        # q i _ i+1
        q_forward_i = la*p*2*x_n*(N-i)/(N*(N-1))
        # q i _ i-1
        q_backward_i = al*i
        # waiting time rate v_i = (q i _ i+1) + (q i _ i-1)
        v_i = q_forward_i + q_backward_i
        t_i = np.random.exponential(v_i)
        # jumping probability to STATE i+1 is (q i _ i+1)/v_i
        jump = np.random.binomial(n=1,p=(q_forward_i/v_i))
        if (jump ==1):
            x_n += 1
        else:
            x_n -= 1
        # increase time    
        t_n = t_n+t_i
        # append Process
        X_n.append(x_n)
        T_n.append(t_n)
        # increment
        k +=1         
    return (X_n,T_n)



In [18]:
# @ C is the hospital capacity
# @ Beta is the target tracking probability
# @ K is the batch size of each estimate
def Y_n(X_n,C,Beta):
    overCapacityCount = 0
    K = len(X_n)
    for i in range(K):
        if X_n[i] > C:
            overCapacityCount+=1
    Y_n = (overCapacityCount/K) - Beta
    return Y_n
    
def update_lambda(_lambda, Y_n, stepsize):
    _lambda -= float(stepsize) * Y_n
    return _lambda

def update_alpha(_alpha, Y_n, stepsize):
    _alpha -= float(stepsize) * Y_n
    return _alpha

# @ x_n initial infected patient
# set initial lambda
# set inital alpha
# @ C is the hospital capacity 
# @ Beta is the target tracking probability 
# @ K is the batch size of each estimate
# these are all set arbitary atm
x_n, _lambda,_alpha,K, C, Beta, N, T, stepsize = (100,
                                                0.5,
                                                0.5,
                                                10,
                                                6000,
                                                0.5,
                                                60000,
                                                30000,
                                                0.001)



trajectory_lambda = []
trajectory_alpha = []
trajectory_X_n = []
# use iteration of 5000 as end of all sequential update
iteration = 0
while iteration < 5000:
    # update alpha
    X_n,_ = simulation_infected(_lambda,0.2,_alpha,N,T,x_n,K)
    print(X_n)
    _alpha = update_alpha(_alpha, Y_n(X_n,C,Beta),stepsize)
    trajectory_alpha.append(_alpha)
    trajectory_X_n.extend(X_n)
    
    # to keep the time consistent, slice the tail of last number of infected
    # as the new initial
    x_n = X_n[-1]
    
    # update lambda
    X_n,_ = simulation_infected(_lambda, 0.2, _alpha, N, T, x_n, K)
    _lambda = update_lambda(_lambda,Y_n(X_n,C,Beta),stepsize)
    
    trajectory_lambda.append(_lambda)
    trajectory_X_n.extend(X_n)
    
    x_n = X_n[-1]
    
    iteration+=1
    

0.001
[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90]
[80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70]
[60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50]
[40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30]
[20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10]


AssertionError: 